### Objective of Data Preprocessing

The goal is to aggregate the movie database to retrieve additional information about the films.

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import pandas as pd
from utils import extract_title_year , remove_punctuation

### Data Import

IMDB and MovieLens are movie databases. They contain information about movies, actors, directors, user ratings, and more.

In [24]:
users_movies_lens = pd.read_csv(
    'data/ml-1m/users.dat',
    sep="::",
    names=["user_id", "sex", "age_group", "occupation", "zip_code"],
    engine="python",
)

ratings_movies_lens = pd.read_csv(
    'data/ml-1m/ratings.dat',
    sep="::",
    names=["user_id", "movie_id", "rating", "unix_timestamp"],
    engine="python",
)

movies_movies_lens = pd.read_csv(
    'data/ml-1m/movies.dat',
    sep="::",
    names=["movie_id", "title", "genres"],
    engine="python",
    encoding="latin-1",
)

ratings_imdb = pd.read_csv('data/title.ratings.tsv', sep='\t')

movies_imdb = pd.read_csv('data/title.basics.tsv', sep='\t')

/var/folders/nm/xmbyqkhd1wjg9j848np2w3gr0000gn/T/ipykernel_56146/538781482.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_imdb = pd.read_csv('data/title.basics.tsv', sep='\t')


In [25]:
users_movies_lens["user_id"] = users_movies_lens["user_id"].apply(lambda x: f"user_{x}")
users_movies_lens["age_group"] = users_movies_lens["age_group"].apply(lambda x: f"group_{x}")
users_movies_lens["occupation"] = users_movies_lens["occupation"].apply(lambda x: f"occupation_{x}")

movies_movies_lens["movie_id"] = movies_movies_lens["movie_id"].apply(lambda x: f"movie_{x}")

ratings_movies_lens["movie_id"] = ratings_movies_lens["movie_id"].apply(lambda x: f"movie_{x}")
ratings_movies_lens["user_id"] = ratings_movies_lens["user_id"].apply(lambda x: f"user_{x}")
ratings_movies_lens["rating"] = ratings_movies_lens["rating"].apply(lambda x: float(x))


We separate the release date and year of the movie to use them in the analyses.

In [26]:
movies_movies_lens[['title', 'year']] = movies_movies_lens['title'].apply(extract_title_year)

We only keep the movies that are in both datasets.

In [27]:
movies_imdb = movies_imdb[movies_imdb['titleType'].isin(['movie', 'tvMovie'])]

Replace the missing values in the year column with the 0 in order to be able to convert the column to an integer.

In [28]:
movies_imdb.startYear = movies_imdb.startYear.replace('\\N', 0)

### Code Description

This code prepares and cleans movie titles from two different databases (IMDb and MovieLens) to make them comparable. Here is a detailed explanation of each step:

1. **Normalization of IMDb movie titles (primaryTitle):**
    - Titles are converted to lowercase.
    - Tab, newline, and carriage return characters are removed.
    - Leading and trailing whitespace is stripped.
    - Punctuation is removed.
    - The article "the" is removed from titles.

2. **Normalization of IMDb movie titles (originalTitle):**
    - Titles are processed similarly to `primaryTitle`, converting to lowercase, removing tab, newline, and carriage return characters, and stripping leading and trailing whitespace.
    - Punctuation is removed.
    - The article "the" is removed from titles.

3. **Normalization of MovieLens movie titles:**
    - Titles are converted to lowercase.
    - Tab, newline, and carriage return characters are removed.
    - Titles are truncated before parentheses (to remove years).
    - Leading and trailing whitespace is stripped.
    - Punctuation is removed.
    - The article "the" is removed from titles.

These steps allow for the comparison and merging of movie titles from both databases by reducing variations due to differences in case, punctuation, and articles.

In [29]:
movies_imdb['title_primary_compared'] = movies_imdb['primaryTitle'].str.lower().str.replace(r'[\t\n\r]', '', regex=True).str.strip()
movies_imdb['title_primary_compared_no_ponct'] = movies_imdb['title_primary_compared'].apply(remove_punctuation)
movies_imdb['title_primary_no_the'] = movies_imdb['title_primary_compared_no_ponct'].astype(str).str.replace('the', '').str.strip()

movies_imdb['title_original_compared'] = movies_imdb['originalTitle'].str.lower().str.replace(r'[\t\n\r]', '', regex=True).str.strip()
movies_imdb['title_original_compared_no_ponct'] = movies_imdb['title_original_compared'].apply(remove_punctuation)
movies_imdb['title_original_no_the'] = movies_imdb['title_original_compared_no_ponct'].astype(str).str.replace('the', '').str.strip()

movies_movies_lens['title_compared'] = movies_movies_lens['title'].str.lower().str.replace(r'[\t\n\r]', '', regex=True).apply(lambda x: x.split('(')[0].strip()).apply(lambda x: x.replace('\n', ''))
movies_movies_lens['title_compared_no_ponct'] = movies_movies_lens['title_compared'].apply(remove_punctuation)
movies_movies_lens['title_no_the'] = movies_movies_lens['title_compared_no_ponct'].astype(str).str.replace('the', '').str.strip()

This code performs additional processing on movie titles from the MovieLens database to handle specific cases where titles may have been stored in a particular format.


1. **Adjust Titles with ", the" Suffix:**
    - The code checks each title in the `title_compared` column for the presence of the suffix ", the".
    - If a title contains ", the", the code modifies the title to move "the" to the beginning of the title.
    - Specifically, it splits the title at the last occurrence of ", " and then reconstructs the title by prepending "the " to the main part of the title.
    - If ", the" is not found in the title, the title remains unchanged.

##### Example
- **Original Title:** "Empire Strikes Back, the"
- **Processed Title:** "the Empire Strikes Back"

This step ensures consistency in how titles starting with "the" are handled, making them more uniform and easier to compare across different datasets.

In [30]:
movies_movies_lens['title_compared'] = movies_movies_lens['title_compared'].apply(lambda x: "the " + x.rsplit(", ", 1)[0] if ", the" in x else x)

These merging operations aim Ensure Robust Matching by using different processed versions of the titles, the code maximizes the chances of correctly matching movie records between the two datasets despite variations in title formatting.

In [31]:
movies_primary = pd.merge(movies_imdb, movies_movies_lens, left_on=['title_primary_compared'], right_on=['title_compared', ])
movies_original = pd.merge(movies_imdb, movies_movies_lens, left_on=['title_original_compared'], right_on=['title_compared', ])

movies_primary_no_ponct = pd.merge(movies_imdb, movies_movies_lens, left_on=['title_primary_compared_no_ponct',], right_on=['title_compared_no_ponct', ])
movies_original_no_ponct = pd.merge(movies_imdb, movies_movies_lens, left_on=['title_original_compared_no_ponct', ], right_on=['title_compared_no_ponct',])

movies_primary_no_ponct_no_the = pd.merge(movies_imdb, movies_movies_lens, left_on=['title_primary_no_the'], right_on=['title_no_the',])
movies_original_no_ponct_no_the = pd.merge(movies_imdb, movies_movies_lens, left_on=['title_original_no_the'], right_on=['title_no_the',])


1. **Concatenation of DataFrames:**
    - `movies`: The various DataFrames resulting from the merges (`movies_primary`, `movies_original`, `movies_primary_no_ponct`, `movies_original_no_ponct`, `movies_primary_no_ponct_no_the`, `movies_original_no_ponct_no_the`) are concatenated into a single DataFrame.

2. **Removal of Duplicates:**
    - Duplicates are removed based on the `tconst` column (unique identifier for movies in IMDb) to ensure there are no duplicate entries in the final dataset.

3. **Filtering by Year:**
    - The movies are filtered to keep only those whose release year (`year`) matches the `startYear` in IMDb. This ensures consistency in the movie release date information.

In [32]:
movies = pd.concat([movies_primary, movies_original, movies_primary_no_ponct, movies_original_no_ponct, movies_primary_no_ponct_no_the, movies_original_no_ponct_no_the])
movies.drop_duplicates(subset=['tconst'], inplace=True)
movies = movies[movies['year'] == movies['startYear']]

In [33]:
print(f"Number of movies in the lens dataset: {len(movies)}")

Number of movies in the lens dataset: 3333


Remove unnecessary intermediate and redundant columns from the final consolidated movie dataset to streamline it for further analysis and modeling.

In [34]:
drop_columns = ['title_primary_compared', 'title_primary_compared_no_ponct', 'title_primary_no_the', 'title_original_compared', 'title_original_compared_no_ponct', 'title_original_no_the', 'title_compared', 'title_compared_no_ponct', 'title_no_the', 'titleType','startYear', 'originalTitle', 'primaryTitle', 'genres_x','endYear']
movies = movies.drop(columns=drop_columns)

Merge the movie dataset with the ratings dataset to include the ratings information for each movie.

In [35]:
movies = pd.merge(movies, ratings_imdb, left_on='tconst', right_on='tconst')

One-hot encode the genres column to convert the categorical data into a numerical format that can be used in machine learning models.

In [36]:
genres = [
    "Action",
    "Adventure",
    "Animation",
    "Children's",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]

for genre in genres:
    movies[genre] = movies["genres_y"].apply(
        lambda values: int(genre in values.split("|"))
    )

Remove the tconst column, which is no longer needed, as the movie identifier is not required for the analysis.

In [37]:
movies = movies.drop(columns=["tconst"])

Reorder the columns in the final dataset to have the movie title and year at the beginning, followed by the ratings information and genres.

In [38]:
new_index = ['movie_id', 'title', 'year', 'genres_y', 'isAdult', 'runtimeMinutes', 'averageRating', 'numVotes', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = movies.reindex(columns=new_index)

Rename the columns ```genres_y``` to ```genres``` to reflect the final dataset's structure.

In [39]:
movies = movies.rename(columns={'genres_y': 'genres'})

Standardize the grades by dividing them by 2 to have a scale from 0 to 5.

In [40]:
movies['averageRating'] = movies['averageRating']/2

#### Save the final movie dataset to a CSV file for further analysis and modeling.

In [41]:
movies.to_csv('data/movies.csv', index=False)